# Pipeline con LangChain: PDF → Teacher (ChatGPT) → JSONL → Student (Ollama/Qwen) → Versionado → Knowledge Base

Este notebook implementa el flujo **mínimo** usando **LangChain**:

1. Extraer texto plano desde un PDF (factura)
2. Llamar a **ChatGPT como teacher** con **salida estructurada** (Pydantic)
3. Guardar artifacts + generar dataset **JSONL** para fine-tuning local (LoRA)
4. Llamar a **Ollama/Qwen como student** (también con salida JSON) y validar con el mismo schema
5. Registrar un **knowledge base log** (JSONL + SQLite opcional)

> Nota: **Ollama no entrena por API**. Aquí generamos el JSONL para entrenar con una tool local (LoRA) y luego cargar el modelo resultante en Ollama.

Generado: 2026-01-05


## 0) Instalación
Instala dependencias principales:
- `langchain`, `langchain-core`
- `langchain-openai` (ChatGPT teacher)
- `langchain-ollama` (Ollama/Qwen student)
- `pdfplumber` para extraer texto
- `pydantic` para schema


In [ ]:
# !pip -q install langchain langchain-core langchain-openai langchain-ollama pdfplumber pydantic requests python-dotenv

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())


## 1) Configuración
- `OPENAI_API_KEY` debe estar en variables de entorno
- Ollama debe estar corriendo en `http://localhost:11434`
- Cambia `PDF_PATH` si lo necesitas


In [ ]:
from pathlib import Path
import os, json, hashlib, datetime
import pdfplumber
from typing import List, Optional
from pydantic import BaseModel, Field

PDF_PATH = Path("facturas_compras_sample01.pdf")

# Teacher (OpenAI via LangChain)
TEACHER_MODEL = os.getenv("TEACHER_MODEL", "gpt-4o-mini")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Student (Ollama via LangChain)
OLLAMA_BASE_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
STUDENT_MODEL = os.getenv("OLLAMA_MODEL", "qwen2.5:7b-instruct")

RUN_ID = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
ARTIFACTS_DIR = Path("artifacts") / RUN_ID
KB_DIR = Path("knowledge_base")
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
KB_DIR.mkdir(parents=True, exist_ok=True)

print("PDF exists?:", PDF_PATH.exists())
print("ARTIFACTS_DIR:", ARTIFACTS_DIR.resolve())
print("KB_DIR:", KB_DIR.resolve())


## 2) Extraer texto del PDF
Si el PDF es escaneado (imagen), `pdfplumber` puede devolver poco texto. Para escaneados, usa OCR.


In [ ]:
def extract_text_pdfplumber(pdf_path: Path) -> str:
    parts = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for page in pdf.pages:
            parts.append(page.extract_text() or "")
    return "\n\n".join(parts).strip()

raw_text = extract_text_pdfplumber(PDF_PATH)
print("chars:", len(raw_text))
print(raw_text[:900])


## 3) Schema Pydantic (teacher/student)
Usaremos el mismo schema para:
- Validar lo que responde ChatGPT (teacher)
- Validar lo que responde Qwen/Ollama (student)


In [ ]:
class SensitivityLabel(BaseModel):
    label: int = Field(..., description="0=no sensible, 1=sensible")
    confidence: float = Field(..., ge=0.0, le=1.0)
    signals: List[str] = Field(default_factory=list)
    rationale_short: str = Field(..., description="Una frase breve, sin pasos intermedios.")

def sha256_text(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

DOC_HASH = sha256_text(raw_text)
DOC_HASH


## 4) Prompt base (teacher)
Forzamos salida estructurada. Con LangChain lo más limpio es usar `with_structured_output(SensitivityLabel)`.


In [ ]:
TEACHER_SYSTEM = (
    "Eres un analista experto en protección de datos. "
    "Clasifica documentos administrativos/financieros por sensibilidad."
)

TEACHER_USER_TEMPLATE = """Clasifica el siguiente documento como sensible o no sensible.

Reglas:
- label=1 si aparece IBAN, NIF/CIF, datos bancarios, datos de pago, identificadores fiscales o información financiera identificable.
- label=0 si no aparecen.

Devuelve SOLO la estructura (no texto extra).

TEXTO:
<<<
{doc_text}
>>>
"""


## 5) Teacher con LangChain (ChatOpenAI) + salida estructurada
Requiere:
- `OPENAI_API_KEY` en env
- `langchain-openai`


In [ ]:
# Teacher (OpenAI) con LangChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

def teacher_label_with_langchain(doc_text: str) -> SensitivityLabel:
    if not OPENAI_API_KEY:
        raise RuntimeError("Falta OPENAI_API_KEY en el entorno.")

    llm = ChatOpenAI(model=TEACHER_MODEL, temperature=0)
    llm_struct = llm.with_structured_output(SensitivityLabel)  # devuelve instancia Pydantic

    prompt = ChatPromptTemplate.from_messages([
        ("system", TEACHER_SYSTEM),
        ("user", TEACHER_USER_TEMPLATE),
    ])

    chain = prompt | llm_struct
    return chain.invoke({"doc_text": doc_text})

# Ejecuta teacher:
# teacher_out = teacher_label_with_langchain(raw_text)
# print(teacher_out.model_dump())

print("Descomenta para ejecutar la llamada real al teacher.")


## 6) Guardar artifacts del teacher (versionado)
Guardamos:
- texto extraído
- label del teacher
- metadata de ejecución (modelo, prompt_version, hash)


In [ ]:
PROMPT_VERSION = "teacher_sensitivity_v1"

def save_teacher_artifacts(doc_text: str, teacher_out: SensitivityLabel) -> None:
    (ARTIFACTS_DIR / "inputs").mkdir(exist_ok=True)
    (ARTIFACTS_DIR / "teacher").mkdir(exist_ok=True)

    (ARTIFACTS_DIR / "inputs" / f"{DOC_HASH}.txt").write_text(doc_text, encoding="utf-8")

    meta = {
        "run_id": RUN_ID,
        "doc_hash": DOC_HASH,
        "pdf_path": str(PDF_PATH),
        "prompt_version": PROMPT_VERSION,
        "teacher_model": TEACHER_MODEL,
        "created_at": datetime.datetime.now().isoformat(),
    }
    (ARTIFACTS_DIR / "teacher" / f"{DOC_HASH}.meta.json").write_text(
        json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8"
    )
    (ARTIFACTS_DIR / "teacher" / f"{DOC_HASH}.label.json").write_text(
        json.dumps(teacher_out.model_dump(), indent=2, ensure_ascii=False), encoding="utf-8"
    )

# Ejemplo:
# save_teacher_artifacts(raw_text, teacher_out)
print("OK. Cuando tengas teacher_out, llama save_teacher_artifacts(raw_text, teacher_out).")


## 7) Dataset JSONL para entrenar el SLM local (student)
Generamos un ejemplo SFT tipo `messages + response`.
Esto lo consumes con tu tool de fine-tuning (LoRA) en local.


In [ ]:
STUDENT_SYSTEM = "Eres un clasificador binario de sensibilidad documental. Responde SOLO JSON válido."
STUDENT_USER_TEMPLATE = """Clasifica sensibilidad (0=no sensible,1=sensible) según:
- 1 si hay IBAN, NIF/CIF, datos bancarios, datos de pago, identificadores fiscales o info financiera identificable.
- 0 si no.
Devuelve SOLO JSON con: label, confidence, signals, rationale_short.

Texto:
<<<
{doc_text}
>>>
"""

def append_jsonl(path: Path, obj: dict) -> None:
    with path.open("a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

def build_sft_example(doc_text: str, teacher_out: SensitivityLabel) -> dict:
    return {
        "messages": [
            {"role": "system", "content": STUDENT_SYSTEM},
            {"role": "user", "content": STUDENT_USER_TEMPLATE.format(doc_text=doc_text)},
        ],
        "response": json.dumps(teacher_out.model_dump(), ensure_ascii=False),
        "meta": {
            "doc_hash": DOC_HASH,
            "teacher_model": TEACHER_MODEL,
            "prompt_version": PROMPT_VERSION,
        },
    }

# Ejemplo:
# train_path = ARTIFACTS_DIR / "student_train.jsonl"
# append_jsonl(train_path, build_sft_example(raw_text, teacher_out))
print("OK. JSONL listo cuando tengas teacher_out.")


## 8) Student con LangChain (ChatOllama) en modo JSON
Usamos `langchain-ollama`.
Para forzar JSON, pasamos `format='json'` como parámetro del modelo (Ollama JSON mode).


In [ ]:
from langchain_ollama import ChatOllama

def student_predict_with_langchain(doc_text: str) -> SensitivityLabel:
    # ChatOllama (Ollama) - JSON mode
    # En Ollama, 'format': 'json' fuerza salida JSON.
    llm = ChatOllama(
        model=STUDENT_MODEL,
        base_url=OLLAMA_BASE_URL,
        temperature=0,
        format="json",
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", STUDENT_SYSTEM),
        ("user", STUDENT_USER_TEMPLATE),
    ])

    chain = prompt | llm
    msg = chain.invoke({"doc_text": doc_text[:4000]})  # opcional truncar
    # msg.content es string JSON
    return SensitivityLabel.model_validate_json(msg.content)

# Ejecuta student:
# student_out = student_predict_with_langchain(raw_text)
# print(student_out.model_dump())

print("Descomenta para ejecutar. Requiere Ollama corriendo y el modelo instalado.")


## 9) Knowledge Base Log (JSONL + SQLite opcional)
Registramos cada ejecución como un 'evento' en una base de conocimiento:
- hash documento
- teacher y student
- versiones
- artifacts


In [ ]:
KB_EVENTS = KB_DIR / "events.jsonl"
KB_DB = KB_DIR / "kb.sqlite"

import sqlite3

def log_kb_event(teacher_out: SensitivityLabel, student_out: Optional[SensitivityLabel] = None, notes: Optional[str] = None) -> dict:
    event = {
        "ts": datetime.datetime.now().isoformat(),
        "doc_hash": DOC_HASH,
        "pdf_path": str(PDF_PATH),
        "teacher": teacher_out.model_dump(),
        "student": student_out.model_dump() if student_out else None,
        "versions": {
            "prompt_version": PROMPT_VERSION,
            "teacher_model": TEACHER_MODEL,
            "student_model": STUDENT_MODEL,
            "ollama_base_url": OLLAMA_BASE_URL,
        },
        "artifacts_dir": str(ARTIFACTS_DIR),
        "notes": notes,
    }
    append_jsonl(KB_EVENTS, event)
    return event

def init_kb_db():
    con = sqlite3.connect(str(KB_DB))
    cur = con.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS events (
        ts TEXT,
        doc_hash TEXT,
        pdf_path TEXT,
        prompt_version TEXT,
        teacher_model TEXT,
        student_model TEXT,
        teacher_json TEXT,
        student_json TEXT,
        artifacts_dir TEXT,
        notes TEXT
    )
    """)
    cur.execute("CREATE INDEX IF NOT EXISTS idx_doc_hash ON events(doc_hash)")
    con.commit()
    con.close()

def insert_kb_db(event: dict):
    con = sqlite3.connect(str(KB_DB))
    cur = con.cursor()
    cur.execute(
        """INSERT INTO events VALUES (?,?,?,?,?,?,?,?,?,?)""",
        (
            event["ts"],
            event["doc_hash"],
            event["pdf_path"],
            event["versions"]["prompt_version"],
            event["versions"]["teacher_model"],
            event["versions"]["student_model"],
            json.dumps(event["teacher"], ensure_ascii=False),
            json.dumps(event["student"], ensure_ascii=False) if event["student"] else None,
            event["artifacts_dir"],
            event.get("notes"),
        ),
    )
    con.commit()
    con.close()

init_kb_db()
print("KB ready:", KB_EVENTS.resolve(), "|", KB_DB.resolve())

# Ejemplo completo (cuando tengas teacher_out y student_out):
# event = log_kb_event(teacher_out, student_out, notes="run demo")
# insert_kb_db(event)


## 10) Ejecución end-to-end (pasos)
1) Extraer texto (ya hecho)
2) Teacher:
   - `teacher_out = teacher_label_with_langchain(raw_text)`
   - `save_teacher_artifacts(raw_text, teacher_out)`
3) Dataset:
   - `append_jsonl(ARTIFACTS_DIR/'student_train.jsonl', build_sft_example(raw_text, teacher_out))`
4) Student:
   - `student_out = student_predict_with_langchain(raw_text)`
5) KB:
   - `event = log_kb_event(teacher_out, student_out)`
   - `insert_kb_db(event)`

Entrenamiento local (LoRA) del student:
- usa el JSONL generado con una tool de fine-tuning.
- luego crea un modelo en Ollama (Modelfile) y repite el paso 4.
